In [44]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6) 

In [45]:
brainstroke_data <- read_csv("https://drive.google.com/uc?export=download&id=1yBiO_qBE9_YBvnEyPe2bazH5ZCOBb1d6")

brainstroke_data

Rows: 4981 Columns: 11
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (5): gender, ever_married, work_type, Residence_type, smoking_status
dbl (6): age, hypertension, heart_disease, avg_glucose_level, bmi, stroke

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>
Male,67,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
Male,80,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
Female,49,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Female,45,1,0,Yes,Govt_job,Rural,95.02,31.8,smokes,0
Male,40,0,0,Yes,Private,Rural,83.94,30.0,smokes,0
Female,80,1,0,Yes,Private,Urban,83.75,29.1,never smoked,0


In [3]:
# # changing chr data to categorical int data (gender, smoking status, work type, residence type)
# # changing dbl data to categorical int data (hypertension, heart disease)
# # removing ever_married column
# # changing Residence_type to residence_type

# brainstroke_data_v1 <- brainstroke_data |>
#     mutate(gender = as_factor(ifelse(gender == 'Male', 1, 2)),
#            smoking_status = as_factor(ifelse(smoking_status == 'never smoked', 1, 
#                                       ifelse(smoking_status == 'formerly smoked', 2, 
#                                       ifelse(smoking_status == 'smokes', 3, 
#                                       ifelse(smoking_status == 'Unknown', 4, 'n/a'))))),
#            work_type = as_factor(ifelse(work_type == 'Private', 1, 
#                        ifelse(work_type == 'Self-employed', 2, 
#                        ifelse(work_type == 'Govt_job', 3, 'n/a')))),
#            Residence_type = as_factor(ifelse(Residence_type == 'Rural', 1, 2)),
#            stroke = as_factor(stroke),
#            heart_disease = as_factor(heart_disease),
#            hypertension = as_factor(hypertension)) |>
#     select(1:4, 6:11) 

# colnames(brainstroke_data_v1)[6] <- 'residence_type'

# brainstroke_data_v1

gender,age,hypertension,heart_disease,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke
<fct>,<dbl>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>
1,67,0,1,1,2,228.69,36.6,2,1
1,80,0,1,1,1,105.92,32.5,1,1
2,49,0,0,1,2,171.23,34.4,3,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
2,45,1,0,3,1,95.02,31.8,3,0
1,40,0,0,1,1,83.94,30.0,3,0
2,80,1,0,1,2,83.75,29.1,1,0


<b> GENDER </b> - male (1), female (2)

<b> SMOKING STATUS </b> - never smoked (1), formerly smoked (2), smokes (3), Unknown (4)

<b> WORK TYPE </b> - private (1), self-employed (2), govt job (3)

<b> RESIDENCE TYPE </b> - rural (1), urban (2)

In [46]:
# changing chr data to categorical data (gender, smoking status, work type, residence type, hypertension, heart disease)
# removing ever_married column
# changed Residence_type and bmi column names

# brainstroke_data_v1 <- brainstroke_data |>
#     mutate(gender = as_factor(gender),
#            smoking_status = as_factor(smoking_status),
#            work_type = as_factor(work_type),
#            Residence_type = as_factor(Residence_type),
#            stroke = as_factor(stroke),
#            heart_disease = as_factor(heart_disease),
#            hypertension = as_factor(hypertension)) |>
#     select(1:4, 6:11) 

# colnames(brainstroke_data_v1)[6] <- 'residence_type'
# colnames(brainstroke_data_v1)[8] <- 'body_mass_index'

# brainstroke_data_v1

brainstroke_data_v1 <- brainstroke_data |>
    mutate(across(c(gender, hypertension:Residence_type, smoking_status:stroke),
                  as_factor)) |>
    select(1:4, 6:11)

colnames(brainstroke_data_v1)[6] <- 'residence_type'
colnames(brainstroke_data_v1)[8] <- 'body_mass_index'

brainstroke_data_v1

gender,age,hypertension,heart_disease,work_type,residence_type,avg_glucose_level,body_mass_index,smoking_status,stroke
<fct>,<dbl>,<fct>,<fct>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>
Male,67,0,1,Private,Urban,228.69,36.6,formerly smoked,1
Male,80,0,1,Private,Rural,105.92,32.5,never smoked,1
Female,49,0,0,Private,Urban,171.23,34.4,smokes,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
Female,45,1,0,Govt_job,Rural,95.02,31.8,smokes,0
Male,40,0,0,Private,Rural,83.94,30.0,smokes,0
Female,80,1,0,Private,Urban,83.75,29.1,never smoked,0


<b> FOR FACTOR DATA - </b> yes (1), no (0) (hypertension, heart disease, stroke)

In [49]:
# 1. any missing data?
not_available <- sum(is.na(brainstroke_data_v1))

not_available


# 2. summarized table 1: stroke counts and proportions
total_obs <- nrow(brainstroke_data_v1)

stroke_proportions <- brainstroke_data_v1 |>
    group_by(stroke) |>
    summarize(stroke_count = n(),
              percentage = (stroke_count / total_obs) * 100)

stroke_proportions


# 3. summarized table 2: gender counts and proportions
gender_proportions <- brainstroke_data_v1 |>
    group_by(gender) |>
    summarize(gender_count = n(),
              percentage = (gender_count / total_obs) * 100)

gender_proportions


# 4. summarized table 3: smoker proportions among those with stroke
stroke_smokers <- brainstroke_data_v1 |>
    filter(stroke == 1) |>
    group_by(smoking_status) |>
    summarize(smoker_count = n(),
              percentage = (smoker_count / total_obs) * 100) 
    
stroke_smokers


# 5. summarized table 4: mean body mass indexes (bmi) of those with/without stroke
stroke_bmi <- brainstroke_data_v1 |>
    group_by(stroke) |>
    summarise(avg_bmi = mean(body_mass_index))

stroke_bmi


# 6. summarized table 5: mean average glucose levels of those with/without stroke
stroke_glucose <- brainstroke_data_v1 |>
    group_by(stroke) |>
    summarise(mean_glucose = mean(avg_glucose_level))

stroke_glucose


# 7. summarized table 6: mean age of those with/without stroke
stroke_age <- brainstroke_data_v1 |>
    group_by(stroke) |>
    summarise(avg_age = mean(age))

stroke_age

[1] 0

stroke,stroke_count,percentage
<fct>,<int>,<dbl>
0,4733,95.02108
1,248,4.97892


gender,gender_count,percentage
<fct>,<int>,<dbl>
Male,2074,41.63823
Female,2907,58.36177


smoking_status,smoker_count,percentage
<fct>,<int>,<dbl>
formerly smoked,70,1.4053403
never smoked,89,1.7867898
smokes,42,0.8432042
Unknown,47,0.9435856


stroke,avg_bmi
<fct>,<dbl>
0,28.40970
1,30.18669


stroke,mean_glucose
<fct>,<dbl>
0,104.5691
1,132.1756


stroke,avg_age
<fct>,<dbl>
0,42.14135
1,67.81984
